In [1]:
using Gridap, Gridap.Geometry, Gridap.Fields
using Gmsh, GridapGmsh
using GLMakie, DelimitedFiles, Interpolations
using LinearAlgebra, SparseArrays, KrylovKit
using ChainRulesCore, Zygote
using PartitionedArrays
using NLopt
using GridapMakie

import Gridap.CellData: Interpolable
import ChainRulesCore: rrule
import Gmsh: gmsh

main_path = "/Users/jayyao/Documents/Research/RamanInFluid/"
include(main_path*"Materials/Materials.jl")
include(main_path*"Module/Mesh_Periodic.jl")
include(main_path*"Module/Helper.jl")
include(main_path*"Module/GridapFE.jl")
include(main_path*"Module/Control.jl")
include(main_path*"Module/Model.jl")
include(main_path*"Module/Objective.jl")

g0_p_optimize (generic function with 5 methods)

In [3]:
material = "Ag"
n_λ, k_λ = RefractiveIndex(material,main_path,true)
# 400, 365, 325, 1600 cm^-1
λ1 = 532
λ2 = 549
# λ1 = 400
# λ2 = 427
nf = sqrt(1.77)
nm1 = n_λ(λ1) + 1im * k_λ(λ1)
nm2 = n_λ(λ2) + 1im * k_λ(λ2)
μ = 1
R = 1e-10
g_flat_surface = 147# 125# 147 #25.25
nm1,nm2

(0.054007220216606505 + 3.42898916967509im, 0.05988059701492538 + 3.5892477611940294im)

In [9]:
hr = (λ1+λ2)/nf/4          # Height of Raman molecule
# Geometry parameters of the mesh
L = 400           # Length of the normal region
hair = 500 + hr       # Height of the air region
hs = 300 + hr         # Height of the source location in air
ht = 200 + hr         # Height of the target location in air
hd = 200          # Height of design domain
hsub = 100        # Height of substrate domain below design domain
dpml = 300        # Thickness of the PML
hrd = (hd, hr)
# Characteristic length (controls the resolution, smaller the finer)
# resol = 30        # Number of points per wavelength
l1 = 20      # Air
l2 = 1       # Design domain
l3 = l1           # PML

meshfile = "geometry.msh"
geo_param = PeriodicGeometry(L, hair, hs, ht, hd, hsub, dpml, l1, l2, l3)
MeshGenerator(geo_param, meshfile)

LHp=(Inf, hair + hd)  # Start of PML for x,y > 0
LHn=(Inf, hsub)       # Start of PML for x,y < 0


ω1 = 2 * π / λ1
phys1 = PhysicalParameters(ω1, nf, nm1, nm1, μ, R, dpml, LHp, LHn, hd)
ω2 = 2 * π / λ2
phys2 = PhysicalParameters(ω2, nf, nm2, nm2, μ, R, dpml, LHp, LHn, hd)
# run(`scp wyao@txe1-login.mit.edu:/home/gridsan/wyao/Research/RamanInFluid/cloud/task1/geometry.msh /Users/jayyao/Documents/Research/RamanInFluid/`)

Info    : No current model available: creating one
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reconstructing periodicity for curve connection 2 - 4
Info    : Reconstructing periodicity for curve connection 5 - 7
Info    : Reconstructing periodicity for curve connection 8 - 10
Info    : Reconstructing periodicity for curve connection 11 - 13
Info    : Reconstructing periodicity for curve connection 14 - 16
Info    : Reconstructing periodicity for curve connection 17 - 19
Info    : Reconstructing periodicity for curve connection 22 - 20
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Me

PhysicalParameters(0.011444781980290685, 1.3304134695650072, 0.05988059701492538 + 3.5892477611940294im, 0.05988059701492538 + 3.5892477611940294im, 1.0, 1.0e-10, 300.0, (Inf, 903.1323390677645), (Inf, 100.0), 200.0)

In [10]:
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = (2, 2)  # Filter radius
β = 32.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

α = 1.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1e-4

# Sum over kx
nkx = 30
nparts = nkx / 2

Bp = true          # Matrix B depend on parameters?
pv = 1

# Foundary constraint parameters
c = (r[1])^4
lw = r[1]
ls = r[1]
ηe = fηe(lw / r[1])
ηd = fηd(lw / r[1])

control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)

gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f, true)


Info    : Reading 'geometry.msh'...
Info    : 45 entities
Info    : 106496 nodes
Info    : 213188 elements
Info    : Done reading 'geometry.msh'


GridapParameters(UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), 184980, BodyFittedTriangulation(), Measure(), Measure(), Measure(), Measure(), Measure(), Measure(), GenericCellField(), Int8[8, 8, 8, 8, 8, 8, 8, 8, 8, 8  …  10, 10, 10, 10, 10, 10, 10, 10, 10, 10], 7)

In [ ]:
using FileIO
import Images: Gray
# specify the path to your local image file
img_path = "Initial/L400-01.png"
img = load(img_path)
data = 1.0 .-Float64.(Gray.(img))
function image_to_function(x, data, Lx, Ly)
    Nx, Ny = size(data)
    if x[2] < 5
        return 1.0
    end
    xi = Nx - Int(round(((x[2]-100)/Lx + 0.5) * Nx))
    yi = Int(round((((x[1]+Ly/2) % Ly)/Ly) * Ny))
    if xi > 0 && xi <= Nx && yi > 0 && yi <= Ny
        return data[xi,yi]
    else
        return 0.0
    end
end

# binit(v) = ∫(v * x->image_to_function(x, data, 200, 150))gridap.dΩ
binit(v) = ∫(v * x->image_to_function(x, data, hd, L))gridap.dΩ
pc_vec = assemble_vector(binit, gridap.FE_P)
p_init = p_extract(pc_vec; gridap)
p_init[p_init .< 0.1] .= 0
p_init[p_init .> 0.1] .= 1

#p_init = ones(gridap.np)
pf_vec = pf_p0(p_init; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh

@show g0_p(p_init; kb=0, phys1, phys2, control, gridap)/L/g_flat_surface
fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(hd))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, hd)
# save("Results/shape.png", fig)
fig

In [ ]:
function p_bowtie(x, d, h, L, hd)
    dx = abs(x[1]) - d / 2 
    dy = abs(x[2] - hd / 2)
    bowtie = 0
    if dx >= 0 && dy <= h / (L - d) * dx
        bowtie = 1
    end
    return bowtie
end

function p_triangle(x, h, L)
    dx = abs(x[1])
    dy = x[2]
    if dy <= (1-2*dx/L)*h
        return 1
    else
        return 0
    end
end

function p_circle(x, radius)
    if x[2]<=5 || (x[1]^2+(x[2]-radius-5)^2)<=radius^2
        return 1
    else
        return 0
    end
end

kb = 0
p_trunc(x, ratio) = x[2] < (ratio * hd) ? 1 : 0
# binitialfunc(v) = ∫(v * x->p_trunc(x, 0.5))gridap.dΩ
# binitialfunc(v) = ∫(v * x->p_bowtie(x, -11, 95, L, hd))gridap.dΩ
# binitialfunc(v) = ∫(v * x->p_triangle(x, 0.8*hd, 2*L))gridap.dΩ
binitialfunc(v) = ∫(v * x->p_circle(x, 80))gridap.dΩ
pc_vec = assemble_vector(binitialfunc, gridap.FE_P)
#A_base = assemble_matrix((u,v)->(∫(u*v)gridap.dΩ), gridap.FE_P, gridap.FE_Q)
#pc_vec = A_base\pc_vec
p_init = p_extract(pc_vec; gridap)
p_init[p_init .< 0.1] .= 0
p_init[p_init .> 0.1] .= 1.0


pf_vec = pf_p0(p_init; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh
fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(hd))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, hd)
@show g0_p(p_init; kb, phys1, phys2, control, gridap)/L/g_flat_surface
fig

In [ ]:
#p_init = ones(gridap.np) * 0.5 #+ rand(gridap.np) * 1e-2
β_list = [8.0, 8.0, 16.0, 16.0, 32.0, 32.0, 32.0]
Q_list = [10.0, 50.0, 100.0, 500.0, 1000.0, 1000.0, 1000.0]
d_list = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 1e-2, 1e-2]*1e2

g_opt = 0
usat = Inf#(10)^2/4
kb1 = 0.5 * ω1
kb2 = 0.5 * ω2
#damp = 1
for bi = 1 : 6
    β = β_list[bi]
    α = 1/(2*Q_list[bi])
    damp = d_list[bi] * 4
    if bi < 9
        c = 0
        control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)
    else
        c = (r[1])^4
        control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)
    end

    if bi == 0
        g_opt, p_opt = g0_p_optimize(p_init, 1e-8, 70, kb1, kb2; phys1, phys2, control, gridap, usat, damp)
    
    else
        g_opt, p_opt = g0_p_optimize([], 1e-8, 70, kb1, kb2; phys1, phys2, control, gridap, usat, damp)
    end
#     if bi == 6
#         g_opt, p_opt = g0_p_optimize(p_init, 1e-12, 70; phys1, phys2, control, gridap)
    
#     else
#         g_opt, p_opt = g0_p_optimize([], 1e-12, 70; phys1, phys2, control, gridap)
#     end
    if isfile("p_opt.value.txt")
        run(`rm p_opt_value.txt`)
    end
    open("p_opt_value.txt", "w") do iop
        for i = 1 : length(p_opt)
            p_temp = p_opt[i]
            write(iop, "$p_temp \n")
        end
    end
    open("g_opt_value.txt", "a") do io
        write(io, "$g_opt \n")
    end
end
@show g_opt

numevals = opt.numevals = 70
numevals = opt.numevals = 

In [ ]:
# Output and view
ImportFromFile = true
N = num_free_dofs(gridap.FE_U)
if ImportFromFile
    #p_max = readdlm("cloud/task5/p_opt_value.txt", Float64)
    p_max = readdlm("p_opt_value.txt", Float64)
    p0 = p_max[:]
#     p_cutoff = 0.5
#     p0[p0.<p_cutoff] .= 0.0
#     p0[p0.>p_cutoff] .= 1.0
else
    p0 = zeros(gridap.np)
end

kb1 = 0
kb2 = 0
usat = Inf#(10)^2/4
damp = 4

pf_vec = pf_p0(p0; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh
A1_mat = MatrixA(pth, kb1; phys=phys1, control, gridap)
b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
u1_vec = A1_mat \ b1_vec
u1h = FEFunction(gridap.FE_U, u1_vec)

A2_mat = MatrixA(pth, kb2; phys=phys2, control, gridap)
o_vec = VectorO(1, 1; gridap)
v2_vec = A2_mat' \ o_vec
v2h = FEFunction(gridap.FE_U, v2_vec)

@show g0_p(p0; kb1, kb2, phys1, phys2, control, gridap, usat, damp)/L/g_flat_surface

fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(hd))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, hd)
save("Results/shape.png", fig)
fig

In [ ]:
# A1_mat = MatrixA(pth, 0; phys=phys1, control, gridap)
# b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
# u1_vec = A1_mat \ b1_vec
# u1h = FEFunction(gridap.FE_U, u1_vec)
maxv = maximum(abs.(u1_vec))/1
maxe = 20#usat
e1=abs2(phys1.nf^2)
e2=abs2(phys1.nm^2)
#fig, ax, plt = plot(gridap.Ω, real(u1h), colormap = :bwr, colorrange=(-maxv, maxv))
#fig, ax, plt = plot(gridap.Ω, abs2(u1h), colormap = :hot, colorrange=(0, maxv^2))
fig, ax, plt = plot(gridap.Ω, 2*(sqrt∘(abs((conj(∇(u1h)) ⋅ ∇(u1h))/(e1 + (e2 - e1) * pth)))), colormap = :hot, colorrange=(0, maxe))
Colorbar(fig[1,2], plt)
#ax.title = "|H|"
ax.title = "|E|"
#ax.aspect = AxisAspect(L/(phys1.LHp[2]+phys1.LHn[2]))
#limits!(ax, -L/2, L/2, -phys1.LHn[2], phys1.LHp[2])
ax.aspect = AxisAspect(L/(hd+hr*1))
limits!(ax, -L/2, L/2, 0, hd+hr*1)
save("Results/Field.png", fig)
fig

In [ ]:
Neth = 50
damp = 4
eth_list = range(1,Neth,step=1)
g_sat = zeros(Neth)
psat = p_init#p0
for ei = 1:Neth
    @show ei
    eth = eth_list[ei]
    usat = eth^2/4
    g_sat[ei] = g0_p(psat; kb=0, phys1, phys2, control, gridap, usat, damp)/L/g_flat_surface
    open("400CircleSat.txt", "a") do io
        write(io, "$(g_sat[ei]) \n")
    end
end


In [ ]:
g1 = readdlm("Results/SatFun/$(λ1)CircleSat.txt")
g2 = readdlm("Results/SatFun/$(λ1)L400Sat.txt")
g3 = readdlm("Results/SatFun/$(λ1)L300Sat.txt")
g4 = readdlm("Results/SatFun/$(λ1)L100Sat.txt")
g5 = readdlm("Results/SatFun/$(λ1)L300Sat10.txt")
eth_list = range(1,50,step=1)

scene, layout = layoutscene(resolution=(1400/2,900/2))
ax = layout[1,1]=Axis(scene)
ms = 3

lin1 = lines!(ax,eth_list,g1[:],linewidth=ms,color=:black)
lin2 = lines!(ax,eth_list,g2[:],linewidth=ms)
lin3 = lines!(ax,eth_list,g3[:],linewidth=ms)
lin4 = lines!(ax,eth_list,g4[:],linewidth=ms)
lin5 = lines!(ax,eth_list,g5[:],linewidth=ms)
#ax.yscale = log10
ax.xlabel="|Eth/E₀|"
ax.ylabel="Power"
#ax.title="Eigenvalue distribution"

# Legend(layout[1,1],[lin1, lin2],["Initial", "Optimized"])
# axislegend(ax, [lin1, lin2, lin3, lin4, lin5],["Optimized Cylinder", "L=400 nm", "L=300 nm", "L=100 nm", "Optimized with |Eth/E₀|=10"], 
#     position = :lt, orientation = :vertical)
save("Results/SatFun/400EF_Eth.png",scene)
scene

In [ ]:
# gvalues = readdlm("gvalue.txt", Float64)
# gavlues = gvalues[:]
include(main_path*"Module/Model.jl")
include(main_path*"Module/Objective.jl")

In [8]:
ptest = rand(gridap.np)
δp = rand(gridap.np)*1e-8
grad = zeros(gridap.np)

kb1 = 0.5 * ω1
kb2 = 0.5 * ω2
usat = 5
damp = 1
# g0 = g0_p(p0, grad; kb, phys1, phys2, control, gridap)
# g1 = g0_p(p0+δp,[]; kb, phys1, phys2, control, gridap)
g0 = g0_p(ptest, grad; kb1, kb2, phys1, phys2, control, gridap,usat,damp)
g1 = g0_p(ptest+δp,[]; kb1, kb2, phys1, phys2, control, gridap,usat,damp)
g1-g0, grad'*δp
#1.609129978462141e8

(9.374402054618258e-8, 9.390311476914822e-8)

In [ ]:
kb = 0.0 * ω2
ptest=zeros(gridap.np)
g0_p(ptest; kb, phys1, phys2, control, gridap)/L

In [ ]:
usat = 10
damp = 10
ptest = zeros(gridap.np)
#LSConstraint(ptest, []; control, gridap), LWConstraint(ptest, []; control, gridap)
g0_p(ptest; kb=0, phys1, phys2, control, gridap, usat, damp)/L, g0_p(ptest; kb=0, phys1, phys2, control, gridap)/L

In [ ]:
Nwave = 100
raman_shift = 17
gwave = zeros(Nwave)
λ1_list = zeros(Nwave)

p_max = readdlm("p_opt_value.txt", Float64)
p_cur = p_max[:]

for li = 1:Nwave
    λ1 = 532 +(li-25) * 5
    λ1_list[li] = λ1
    λ2 = λ1 + raman_shift
    nm1 = n_λ(λ1) + 1im * k_λ(λ1)
    nm2 = n_λ(λ2) + 1im * k_λ(λ2)
    ω1 = 2 * π / λ1
    phys1 = PhysicalParameters(ω1, nf, nm1, nm1, μ, R, dpml, LHp, LHn, hd)
    ω2 = 2 * π / λ2
    phys2 = PhysicalParameters(ω2, nf, nm2, nm2, μ, R, dpml, LHp, LHn, hd)
    gwave[li] = g0_p(p_cur, []; kb=0, phys1, phys2, control, gridap)/control.Amp
end

scene, layout = layoutscene(resolution=(1400,900))
ax = layout[1,1]=Axis(scene)

lin1=lines!(ax,λ1_list, gwave[:])
# grad = zeros(gridap.np)
# g0_p(p0, grad; kb, phys1, phys2, control, gridap)
#ax.yscale = log10
ax.xlabel="λ"
ax.ylabel="Power"
#ax.title="Eigenvalue distribution"

# Legend(layout[1,1],[lin1, lin2],["Initial", "Optimized"])
# axislegend(ax, [lin1, lin2, lin3],["All","Far field","Difference"], position = :lt,
#     orientation = :vertical)
# save("Results/EigenValDistributionIO.png",scene)
scene

In [ ]:
Nperiod = 31
usat = (10)^2/4
damp = 4
gmax = zeros(Nperiod)
rmax = zeros(Nperiod)
for li = 1:Nperiod
    L = 100 + (li-1) * 10
    geo_param = PeriodicGeometry(L, hair, hs, ht, hd, hsub, dpml, l1, l2, l3)
    MeshGenerator(geo_param, meshfile)
    gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f, true)

    for radius = 5:Int64(L/2)
        binitialfunc(v) = ∫(v * x->p_circle(x, radius))gridap.dΩ
        pc_vec = assemble_vector(binitialfunc, gridap.FE_P)
        p_init = p_extract(pc_vec; gridap)
        p_init[p_init .< 0.1] .= 0
        p_init[p_init .> 0.1] .= 1
        gtemp = g0_p(p_init; kb=0, phys1, phys2, control, gridap, usat, damp)/L/g_flat_surface
        if gtemp > gmax[li]
            gmax[li] = gtemp
            rmax[li] = radius
        end
    end
    @show L, gmax[li], rmax[li]
end
rmax

In [ ]:
gmax = 0
dmax = 0
hmax = 0
for d = -19:1:1
    for h = 91:1:109
        bowtie_temp(v) = ∫(v * x->p_bowtie(x, d, h, L, hd))gridap.dΩ
        pc_vec = assemble_vector(bowtie_temp, gridap.FE_P)
        p_init = p_extract(pc_vec; gridap)
        p_init[p_init .< 0.1] .= 0
        p_init[p_init .> 0.1] .= 1
        @show gtemp = g0_p(p_init; kb=0, phys1, phys2, control, gridap)/L/g_flat_surface
        if gtemp > gmax
            gmax = gtemp
            dmax = d
            hmax = h
        end
    end
end
gmax, dmax, hmax
# [100,1,180]

In [ ]:
Ng = 40
g_list = zeros(Ng)
g0_list = zeros(Ng)
g1_list = zeros(Ng)
h_list = zeros(Ng)

for hi = 1:Ng
    print(hi,"\n")
    λ2 = 549/sqrt(1.77)
    hr = (hi+0) * λ2 / 2
    h_list[hi] = hr/λ2
    gp, g0= g_hr(hr)
    # gp, g0, g1= g_hr(hr)
    g_list[hi] = gp
    g0_list[hi] = g0
    # g1_list[hi] = g1
    # g_list[hi] = g_hr_fixmesh(pth, hr; phys1, phys2, control, gridap)
end

In [ ]:
sat_test(x, sat, damp) = 1 ./ (1 .+ exp.(damp*(x.-sat)))
xx = range(1,50,step=1)
lines(xx, sat_test(xx.^2, 20^2, 4))